# Notes

1. be sure to use exponential RELU
2. dataset includes 4 classes, 9 test sujects
3. expect no spacial interdependency between channels
4. expect interdependacny in time-domain. Use Conv nets to get frequencies
5. input is 2D-array, where height is the umber of channels, width is time
6. aim is 76% accuracy
7. We do not even try hybrid/residual nets because it was said that it performs worse than Deep net

Simle NN:

We applied batch normalization,
as recommended in the original paper (Ioffe and Szegedy, 2015), to the output of convolutional
layers before the nonlinearity. Dropout randomly sets some inputs for a layer to zero in each
training update. It is meant to prevent co-adaption of different units and can be seen as analogous
to training an ensemble of networks. We drop out the inputs to all convolutional layers after the
first with a probability of 0.5. Finally, our new tied loss function is designed to further regularize
our cropped training (see Section 2.5.4 for an explanation).


Our deep ConvNet had four convolution-max-pooling blocks, with a special first block designed
to handle EEG input (see below), followed by three standard convolution-max-pooling blocks and
a dense softmax classification layer

Dropout for my conv layer is p=0.8 and for my fc layer is p=0.5. Interestingly, I saw the opposite trend where a low dropout number (I tried 0.2 out of curiosity) made overfitting worse and dropped my validation accuracy by 10-15%. Beside dropout, I didn't add any other regularization since that mainly helps with overfitting I believe. 



LET"S TRY TO HAVE TIMESTAMP AS JUST A REGULAR PARAMETER OF THE NETWORK
Idea 1 - let's try to have timestamp as just a regular parameter of the network - I can't perform convolutions in this case - after the first convolution number of channels and numer of samplesmesses up

Idea 2 - let's try to pass the data without additional windowing. Seem Ok at a first glance but not sure whether state of the LSTM cell is really perserved - it seems that addition of LSTM does not help to imporve accuracy

Idea 3 - let's try tp window the data first. Then let's reshape first to dimensions so batch size will be number_of_trials * number_of_windows_in_the_trial. Let's specify statefullness of the cell and make batch_size=number_of_windows_in_the_trial


# Set up the environment

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# import tf
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K

# import os functions
import os
import time

import numpy as np
import matplotlib.pyplot as plt

from IPython import display

# Load the data

### Read the files

In [3]:
X_test = np.load("./EEG_data/X_test.npy")
y_test = np.load("./EEG_data/y_test.npy") - 769
person_train_valid = np.load("./EEG_data/person_train_valid.npy")
X_train_valid = np.load("./EEG_data/X_train_valid.npy")
y_train_valid = np.load("./EEG_data/y_train_valid.npy") - 769
person_test = np.load("./EEG_data/person_test.npy")

print("training/Valid data shape: {}".format(X_train_valid.shape))       # training data of many persons
print("Test data shape: {}".format(X_test.shape))                        # test data of many persons
print("Training/Valid target shape: {}".format(y_train_valid.shape))     # training labels of many persons
print("Test target shape: {}".format(y_test.shape))                      # test labels of many persons
print("Person train/valid  shape: {}".format(person_train_valid.shape))  # which person correspond to the trail in test set
print("Person test shape: {}".format(person_test.shape))                 # which person correspond to the trail in test set

### divide dataset into training and validation

In [5]:
perm = np.random.permutation(X_train_valid.shape[0])
num_train = int(0.8 * X_train_valid.shape[0])
num_valid = X_train_valid.shape[0] - num_train
X_train =  X_train_valid[perm[0:num_train]]
y_train =  y_train_valid[perm[0:num_train]]
X_valid = X_train_valid[perm[num_train: ]]
y_valid = y_train_valid[perm[num_train: ]]


print("Training data shape: {}".format(X_train.shape))
print("Training label shape: {}".format(y_train.shape))
print("Validation data shape: {}".format(X_valid.shape))
print("Validation label shape: {}".format(y_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Test label shape: {}".format(y_test.shape))

Training data shape: (1692, 22, 1000)
Training label shape: (1692,)
Validation data shape: (423, 22, 1000)
Validation label shape: (423,)
Test data shape: (443, 22, 1000)
Test label shape: (443,)


### 1 person dataset

In [6]:
person_num = 0
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

Training data shape for 1 person: (189, 22, 1000)
Training label shape for 1 person: (189,)
Validation data shape for 1 person: (48, 22, 1000)
Validation label shape for 1 person: (48,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)


### Augmented dataset

In [102]:
def sliding_window(X_arr, y_arr, time_window=100, time_step=1, time_stride=1):
    temp_x = np.moveaxis(X_arr, 2, 0)
    temp_x = temp_x.astype(np.float32)
    buff = []
    
    num_slices = (len(temp_x)-time_window*time_step) // time_stride + 1
    
    # get time slices for data
    for i in range(num_slices):
        buff.append(temp_x[i*time_stride:i*time_stride + time_window*time_step:time_step])
        buff[i] = np.moveaxis(buff[i], 0, 2)
        # uncomment this if additional dimension is needed
        # buff[i] = buff[i].reshape(1, buff[i].shape[0], buff[i].shape[1], buff[i].shape[2])
        
    temp_x = np.concatenate(buff)
        
    # get time slice for labels
    temp_y = np.ones((X_arr.shape[0],num_slices))
    
    for i in range(len(y_arr)):
        temp_y[i] = temp_y[i] * y_arr[i]
        
    temp_y = temp_y.reshape((-1))
    
    return temp_x, temp_y

In [214]:
TIME_WINDOW = 1000               # number of time stamps in the windpw
TIME_STEP = 1                   # reverese to frequency
TIME_STRIDE = 300                # stride as in convolution
NUM_OF_WINDOWS = (1000-TIME_WINDOW*TIME_STEP)//TIME_STRIDE + 1

# normilize the data USING ONLY TRAIN DATA MEAN AND STANDARD DEVIATION
X_train_norm = (X_train - np.mean(X_train))/np.std(X_train)
X_valid_norm = (X_valid - np.mean(X_train))/np.std(X_train)
X_test_norm = (X_test - np.mean(X_train))/np.std(X_train)

# cut the slices
X_train_slices, y_train_slices = sliding_window(X_train_norm, 
                                                y_train, 
                                                time_window=TIME_WINDOW, 
                                                time_step=TIME_STEP, 
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(X_valid_norm, 
                                                y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_step=TIME_STEP, 
                                                time_stride=TIME_STRIDE)


X_test_slices, y_test_slices = sliding_window(X_test_norm, 
                                              y_test, 
                                              time_window=TIME_WINDOW, 
                                              time_step=TIME_STEP, 
                                              time_stride=TIME_STRIDE)


print("Training data shape with slices: {}".format(X_train_slices.shape))
print("Training label shape with slice: {}".format(y_train_slices.shape))
print("Validation data shape with slices: {}".format(X_valid_slices.shape))
print("Validation label shape with slice: {}".format(y_valid_slices.shape))
print("Test data shape with slices: {}".format(X_test_slices.shape))
print("Test label shape with slice: {}".format(y_test_slices.shape))

Training data shape with slices: (1692, 22, 1000)
Training label shape with slice: (1692,)
Validation data shape with slices: (423, 22, 1000)
Validation label shape with slice: (423,)
Test data shape with slices: (443, 22, 1000)
Test label shape with slice: (443,)


# Naive RNN model

### Construct model

In [7]:
keras.backend.clear_session()

# input
naive_rnn_input = layers.Input(shape=(22, 1000))
# conv across channels?
p1 = layers.Permute((2, 1))(naive_rnn_input)
lstm1 = layers.LSTM(128, return_sequences=True)(p1)
lstm2 = layers.LSTM(128, return_sequences=True)(lstm1)
d2 = layers.Dense(128, activation="elu")(lstm2)
f2 = layers.Flatten()(d2)

# output
naive_rnn_output = layers.Dense(4, activation="softmax")(f2)

### Compile model

In [8]:
# implement learning rate decay
initial_learning_rate = 1e-3
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                          decay_steps=1692,
                                                          decay_rate=0.5,
                                                          staircase=True)

# save model with the best accuracy 
naive_rnn_callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/naive_rnn',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]


naive_rnn_model = keras.Model(inputs = naive_rnn_input, outputs = naive_rnn_output, name="naive_rnn_model")
naive_rnn_model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), 
                        loss="sparse_categorical_crossentropy", 
                        metrics=["acc"])

In [9]:
naive_rnn_model.summary()

Model: "naive_rnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 22, 1000)]        0         
_________________________________________________________________
permute (Permute)            (None, 1000, 22)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 1000, 128)         77312     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000, 128)         131584    
_________________________________________________________________
dense (Dense)                (None, 1000, 128)         16512     
_________________________________________________________________
flatten (Flatten)            (None, 128000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)             

### Train model

In [14]:
naive_rnn_model_loss_hist = naive_rnn_model.fit(X_train, y_train,
                                                validation_data = (X_valid, y_valid),
                                                epochs = 3,
                                                callbacks=naive_rnn_callbacks)

NameError: name 'naive_rnn_callbacks' is not defined

In [ ]:
best_rnn_model = keras.models.load_model('./model_checkpoints/naive_rnn')

best_rnn_model_results = best_rnn_model.evaluate(X_test, y_test)

print('Naive RNN model test loss:', best_rnn_model_results[0])
print('Naive RNN model test acc:', best_rnn_model_results[1])

# Augmented RNN model

It is clear that naive RNN model suffers from overfit - training loss of the model continues to drop while validation loss saturates and accuracy does not go beyond 50%. This can be fixed in 3 ways - dataset augmentation, architecture improvements and regularization.

1. Architecture improvements: add dropouts to our layers; try GRU instead of LSTM;
2. Dataset augmentation: normilize inputs, decrease the length of the example and upsample using sliding window approach - that produces correlation between examples, but if traning/validation/test sets are separated beforehand, that should be fine.
3. Regularizations: 

Results:

1. Dropout layers do not help to improve accuracy. It stays around 45% anyway
2. Augmentation allowed to achieve validation accuracy of 55% and test accuracy of 50% if std not squared. Trained for 2 epoch. If std is squared, 2 epochs give 52% validation and 50% test. After this accuracy satruates. From this moment on all data is presumed to be normilized without square.
3. Usage of sliding window lead to dramatic decrease of accuracy. From this point on it is assumed that no sliding wndow is applied.
4. a) With length of 1000 we can achieve 56% validation 54% test over 5 epochs. After this learning saturates.
4. b) Decrease of the length to 800 allowed to achieve 58% validation 55% test over 5 epochs. After this learning saturates.
4. c) Decrease of the length to 750 allowed to achieve 59% validation 53% test over 3 epochs. After this learning saturates. 
4. d) Decrease of the length to 600 allowed to achieve 65% validation 57% test over 5 epochs. After this learning saturates. 
4. e) Overall, the gain from smaller window is also small +/- 3% at best. This is why from now we will rn experiments with 1000 but in the end also try 750%.

5. When network starts to overfit you can decrease learning rate to 1e-4 to get a couple of additional persents.
6. Form this point on we use size of 1000 samples per example if reverse is not specified.
7. a) Added regularization to hidden layer of 0.001. Validation 56% test 53%
7. b) Added regularization to hidden layer of 0.1.

### Construct model

In [258]:
keras.backend.clear_session()

# input
aug_rnn_input = layers.Input(shape=(22, TIME_WINDOW))
# conv across channels?
p1 = layers.Permute((2, 1))(aug_rnn_input)
lstm1 = layers.LSTM(64, 
                    return_sequences=True, 
                    dropout=0.3, 
                    kernel_regularizer=keras.regularizers.l2(0.005),
                    activity_regularizer = keras.regularizers.l2(0.005),
                    recurrent_regularizer=keras.regularizers.l2(0.005))(p1)
lstm2 = layers.LSTM(64, 
                    return_sequences=True, 
                    dropout=0.3,
                    kernel_regularizer=keras.regularizers.l2(0.005),
                    activity_regularizer = keras.regularizers.l2(0.005),
                    recurrent_regularizer=keras.regularizers.l2(0.005))(lstm1)


f2 = layers.Flatten()(lstm2)
do2 = layers.Dropout(0.3)(f2)
elu2 = layers.Dense(128, activation="elu",  kernel_regularizer=keras.regularizers.l2(0.005))(do2)

# output
aug_rnn_output = layers.Dense(4, activation="softmax")(elu2)### Construct model

### Compile model

In [259]:
# save model with the best accuracy 
aug_rnn_callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/augmented_rnn',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)### Compile model
]


aug_rnn_model = keras.Model(inputs = aug_rnn_input, outputs = aug_rnn_output, name="augmented_rnn_model")
aug_rnn_model.compile(optimizer=keras.optimizers.Adam(), 
                      loss="sparse_categorical_crossentropy", 
                      metrics=["acc"])

In [260]:
aug_rnn_model.summary()

Model: "augmented_rnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 22, 1000)]        0         
_________________________________________________________________
permute (Permute)            (None, 1000, 22)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 1000, 64)          22272     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000, 64)          33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 1000, 64)          33024     
_________________________________________________________________
flatten (Flatten)            (None, 64000)             0         
_________________________________________________________________
dropout (Dropout)            (None, 64000)     

### Train model

In [261]:
num_epochs += 1
train_shuffle = np.arange(len(X_train_slices))
np.random.shuffle(train_shuffle)
aug_rnn_model_loss_hist = aug_rnn_model.fit(X_train_slices[train_shuffle], y_train_slices[train_shuffle],
                                            validation_data = (X_valid_slices, y_valid_slices),
                                            epochs = 2,
                                            batch_size=10
                                            # due to yet unresolved tf bug, we can not save augmented model in a callback
                                            # https://github.com/tensorflow/tensorflow/issues/32505
                                            #callbacks=aug_rnn_callbacks
                                           )

Train on 1692 samples, validate on 423 samples
Epoch 1/2
1692/1692 [==============================] - 241s 142ms/sample - loss: 6.7103 - acc: 0.3836 - val_loss: 2.7707 - val_acc: 0.3570
Epoch 2/2
1692/1692 [==============================] - 230s 136ms/sample - loss: 2.8565 - acc: 0.4456 - val_loss: 2.0907 - val_acc: 0.5366


In [245]:
aug_rnn_model_results = aug_rnn_model.evaluate(X_test_slices, y_test_slices)

print('Naive RNN model test loss:', aug_rnn_model_results[0])
print('Naive RNN model test acc:', aug_rnn_model_results[1])

443/443 [==============================] - 6s 13ms/sample - loss: 1.8969 - acc: 0.5350
Naive RNN model test loss: 1.8969259714165336
Naive RNN model test acc: 0.5349887


In [178]:
print(num_epochs)

9
